In [1]:
import sys

sys.path.append("../")

import numpy as np
import joblib 
import os

from terminal.Extensions.MT5Ext import MT5Ext

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
ticks = np.load("./data/ticks_WIN$N_2024_6.npz", allow_pickle=True)["ticks"]

In [3]:
data = MT5Ext.create_ohlc_from_ticks(ticks, '2s')

In [4]:
# Pré-processamento de dados
data["CURRENT_CHANGE"] = data["close"].pct_change()
data["FORECAST_CHANGE"] = data["CURRENT_CHANGE"].shift(-1)
data["TARGET"] = np.where(
    (data["CURRENT_CHANGE"] > 0) & (data["FORECAST_CHANGE"] > 0)
    | (data["CURRENT_CHANGE"] < 0) & (data["FORECAST_CHANGE"] < 0),
    1,
    0,
)

data.dropna(inplace=True)

In [5]:
# Criar características e rótulos
feature_columns = ["CURRENT_CHANGE"]

bought_data = data[data["close"] > data["open"]]
sold_data = data[data["close"] < data["open"]]

bought_features = bought_data[feature_columns]
bought_target = bought_data["TARGET"]

sold_features = sold_data[feature_columns]
sold_target = sold_data["TARGET"]

In [6]:
# Dividir em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(bought_features, bought_target, test_size=0.2, random_state=42)

# Treinar o modelo
if os.path.exists("../models/bought.joblib"): 
    model = joblib.load("../models/bought.joblib")
else:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'bought accuracy: {accuracy:.2f}')

# save
joblib.dump(model, "../models/bought.joblib")

bought accuracy: 0.76


['../models/bought.joblib']

In [7]:
# Dividir em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(sold_features, sold_target, test_size=0.2, random_state=42)

# Treinar o modelo
if os.path.exists("../models/sold.joblib"): 
    model = joblib.load("../models/sold.joblib")
else:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'sold accuracy: {accuracy:.2f}')

# save
joblib.dump(model, "../models/sold.joblib")

sold accuracy: 0.77


['../models/sold.joblib']